In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait() 

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

In [5]:
%%time
####### CHOIX DE LA TIME FRAME
_period = 'm15'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = list(sorted(set(get_ticker_list())))
TIK = ['AUD','NZD','GBP','JPY','CHF','CAD','SEK','NOK','ILS','MXN','USD','EUR']
RATE = [0.776,0.721,1.3912,1/105.91,1/0.892,1/1.2681,1/8.2884,1/8.4261,1/3.2385,1/20.1564,1,1/1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################
df_all = joblib.load('JOBLIB/Built_bases/df_all')
daily_all = joblib.load('JOBLIB/Built_bases/daily_all')
weekly_all = joblib.load('JOBLIB/Built_bases/weekly_all')
print(df_all.shape,daily_all.shape,weekly_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2016-08-31' # stop the train and begin the test there '2016-08-31'
_stop = '2017-12-31' # stop the test there. After that, it is kept for oos
_last = '2020-12-31' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[19]
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000 / df_ratefx.loc[x[:3],'rate']
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = joblib.load('JOBLIB/Built_bases/df_all')
df_all = stochastic(df_all)
features = featuring(df_all)
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

# Proceed an MaxAbsScaler on features
features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

from sklearn.neural_network import MLPClassifier
_name = 'MLPClassifier'
_model = MLPClassifier(hidden_layer_sizes=(200,8), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]
savename = 'JOBLIB/'+_name+'/Save_'+x.replace('/','')+'_m15.sav'
joblib.dump(_model, savename)
print(col.Fore.BLUE,"Joblib Model ",savename," dumped to disk",col.Style.RESET_ALL)
print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)].dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))

#df_all_oos = df_all_oos.dropna()

df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

print(x)
_year_bottom = _stop
_year_top = _last

#_target =  0.0015
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)


Récupération des tickers
(11069853, 56) (119160, 7) (23981, 19)
GBP/CAD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-01 10:53:10.688473

Chargement de la nouvelle base


 Le rate du ticker GBP/CAD est à  0.7885813421654444 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 72603/72603 [00:02<00:00, 26672.56it/s]
 For ticker  GBP/CAD 
 
Total Profit & Loss : $  -82958.62 . En  1466   transactions.
 
Winners Number : 432 
 
Loosers number : 1034 
BT's execution time 0:00:09.010423
 GBP/CAD  results 
 Tested Period 2017-12-31  à 2020-12-31 
 Total Number of trades 1466 
Started Cash : 200000
P&L in currency:  -82958.62$ 
P&L in %:  -41.48% 
Average trade 

In [3]:
len(features_oos),len(df_all_oos),len(features_oos.dropna()),len(df_all_oos.dropna())

(72328, 72328, 72328, 72328)

In [ ]:
ID = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NOK','EUR/NZD','EUR/SEK','EUR/TRY','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/NZD','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','TRY/JPY','USD/CAD','USD/CHF','USD/JPY','USD/MXN','USD/NOK','USD/TRY','USD/ZAR']

PNL1 = [15914.97,23150.52,25749.05,29127.9,27656.5,30287.02,17955.41,18530.83,73189.81,30495.69,17792.3,29292.47,27452.09,20720.86,99477.85,5733.53,18500.5,41341.0,70165.45,55842.08,63097.12,22576.85,89258.44,43995.5,17399.69,12873.51,21046.02,28376.0, -647.06,41249.22,25043.94,25090.61,31031.38,33484.01,64753.84,47593.89]#,18976.28,]

PNL2 = [16598.35,20327.62,25591.08,33675.18,31577.5,29354.21,20582.07,18976.28,82064.88,27842.08,19697.41,32083.24,24092.98,22133.61,92350.37,12832.98,14707.54,43019.5,70735.41,56800.55,55308.15,27495.73,99733.37,42449.5,20835.03,10529.54,22232.92, 28922.5,-931.69,41022.73,26474.02,30264.23,34768.32,28487.47,17428.88,47893.86]

In [ ]:
# pb => USD/CNH, HKD =>pas bon, USD/SEK => plante, XAG/USD => mauvais, XAU/USD => bon, mais facteur 1000?, ZAR/JPY => plante
res = pd.DataFrame(index=ID)
res['PnL_tp_sl_inv'] = PNL1
res['PnL_tp_sl_no_inv'] = PNL2
res.PnL_tp_sl_inv.sum(),res.PnL_tp_sl_no_inv.sum()

In [7]:
res.sort_values('PnL_tp_sl_no_inv',ascending=False)

,PnL_tp_sl_inv,PnL_tp_sl_no_inv
GBP/NZD,89258.44,99733.37
EUR/NZD,99477.85,92350.37
EUR/AUD,73189.81,82064.88
GBP/AUD,70165.45,70735.41
GBP/CAD,55842.08,56800.55
GBP/CHF,63097.12,55308.15
USD/ZAR,47593.89,47893.86
EUR/USD,41341.00,43019.50
GBP/USD,43995.50,42449.50
USD/CAD,41249.22,41022.73


In [6]:
len(df_all[df_all.Symbol=='EURUSD'])

265811

Timestamp('2013-07-14 00:00:00')

In [9]:
15*265811

3987165

In [19]:

_fin

Timestamp('2021-02-10 00:00:00')

In [22]:
df_all.shape

(11069853, 57)

In [5]:
TICKER_LIST

['AUD/CAD',
 'AUD/CHF',
 'AUD/JPY',
 'AUD/NZD',
 'AUD/USD',
 'CAD/CHF',
 'CAD/JPY',
 'CHF/JPY',
 'EUR/AUD',
 'EUR/CAD',
 'EUR/CHF',
 'EUR/GBP',
 'EUR/JPY',
 'EUR/NOK',
 'EUR/NZD',
 'EUR/SEK',
 'EUR/TRY',
 'EUR/USD',
 'GBP/AUD',
 'GBP/CAD',
 'GBP/CHF',
 'GBP/JPY',
 'GBP/NZD',
 'GBP/USD',
 'NZD/CAD',
 'NZD/CHF',
 'NZD/JPY',
 'NZD/USD',
 'TRY/JPY',
 'USD/CAD',
 'USD/CHF',
 'USD/CNH',
 'USD/HKD',
 'USD/JPY',
 'USD/MXN',
 'USD/NOK',
 'USD/SEK',
 'USD/TRY',
 'USD/ZAR',
 'XAG/USD',
 'XAU/USD',
 'ZAR/JPY']